# Project summary:

This project was initally inspired by [bol.com's serenade project](https://github.com/bolcom/serenade) which offers a fast implementation of the [V-SKNN algorithm](https://arxiv.org/abs/1803.09587), able to serve online customers with personalised recommendations.  
The key figures highlighted in the serenade paper show a P90 latency of 7ms and a throughput of 600 requests per second. Serenade reviews up to 5000 historical sessions (neighbours) per item in the current session (up to 10).    
The serenade implmentation runs many computations sequentially, while many of those have no interdependencies, suggesting that better performances could be reached with GPU paralelization.  
This is where the [NVIDIA RAPIDS](https://developer.nvidia.com/rapids) suite of libraries come into play.  This project offers a GPU implementation of V-SKNN aiming to reach similar or better performances than Serenade.


## Algorithm

### V-sKNN short example:
Let's say the user has a session comprising to three ordered items `[item_1, item_2, item_3]`, with `item_3` being the most recent. The algorithm will perfom the following steps:
* **Lookup candiates (neighbor) sessions:** The algorithm is going to retrieve the historical sessions that contain the item *i* for each item in the input session. **hyperparameter** number of historical sessions per item, `max_sessions_per_item` set to 5000 in this example  
* **Calculate the historical session similarity:** This is a step that we want to paralelise as session similarities do not depend on each other. There are many different methods that could be considered here, but one of the key feature of V-sKNN is that each item in the input session carries a different weight. In this example we use a linear decay with weights of `0.33, 0.66` and `1.0` for items  `item_1`, `item_2` and `item_3` respectively. Each session similarity is calcuated as the sum of weights of items that are also found in the input sessions.  **hyperparameter:** weight functions. We use a linear deay in this example.
* **Keep the top-k most similar sessions:** The top K most similar sessions are kept and the rest is ignored.  **hyperparameter:** value of `top_k`, set to 100 in this example.
* **Calculate item similarity:** each items appearing in the K most similar session is considered for recommendation. The score of each item is the sum of the similarities of sessions that contain the item.  

The same "similarity" method is used twice: First to calculate session similarities using item weights, then to calculate item similarities using session weights. The same function is re-used in both cases.  

### GPU implementation challenges and solutions

* **indexing historical data:** Implementations of V-sKNN such as Serenade assume the presence of an efficient key-value store to retrieve the historical session data.  This was experimented with in previous iterations of this project (and could remain a valid option), but the transfer of data from host to device became one of the algorithm's main bottleneck.  So effort was put into making sure that the historical data can be kept and efficiently read on the device.  The current solution is convert the `item_id` and `session_id` into contiguous integers, so that the data for item (session) *i* can be found at the *i-th* row of an array. In order to also keep the memory usage reasonable, we cannot use a 2-D array. Instead, the index *i* enables to lookup the starting and ending position of the historical data for item (session) *i* in another array which contains the values (no padding necessary). A [CuPy RawKernel](https://docs.cupy.dev/en/stable/reference/generated/cupy.RawKernel.html) was created for this operation: It copies the necessary values from the historical data stored on device, into a temporary "buffer". 
* **similarity function:** The similarity of sessions (items) can be loosely described as a "group by and sum" operation. However, a cudf groupby was giving performances that were below this proejct's targets, another CuPy `RawKernel` was created here for a groupby operation on the previously mentioned values "buffer". 
* **unique items:** For the "groupby" kernel to work, we need a list of unique items (sessions). [CuPy unique](https://docs.cupy.dev/en/stable/reference/generated/cupy.unique.html?highlight=unique#cupy.unique) function does not yet support the `axis` option. A quick rip-off of the CuPy function was built for the occasion. This project can also be an opportunity to add the support for `axis` in CuPy unique.
* **converting the item (session) values to integer idx**. At the moment this is done with a simple python dictionary, but could be improved.

### Validation:
The algorithm was validated by replicating the results found in https://github.com/rn5l/session-rec


In [51]:
import cudf
import os
import cupy as cp
import numpy as np
from vs_knn.vs_knn import CupyVsKnnModel
import gc
import time
from tqdm import tqdm
import sys

## Dataset: RSC15 "*Yoochoose*" data
The public *RSC15* e-commerce dataset is one of the largest publicly available e-commerce datasets, which makes it a good choice for illustrating the present project

In [ ]:
!pip install wget  # wget missing from the docker image

In [ ]:
from wget import download

In [ ]:
dataset_filepath = 'archive/yoochoose-clicks.dat'

def bar_progress(current, total, width=80):
    progress_message = "Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total)
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()
    

if not os.path.isfile(dataset_filepath):
    download("https://storage.googleapis.com/jcrousse-vsknn-rapids/yoochoose-clicks.dat", out=dataset_filepath, bar=bar_progress)

Downloading: 99% [1480630272 / 1486798186] bytes

## Data Loading
The algorithm expects to receive a cudf dataframe with 3 columns: `item_id`, `session_id` and `timestamp`. The order does not matter.  
The algorithm also does works with int or string `item_id` and `session_id`. We could aim to improve the performances by putting more restrictions on the input format, but at this stage we prefer a solution that is more flexible.  
Different public datasets may require a different loading function. Here we implement one for the original RSC15 dataset to showcase an end-to-end example 

In [52]:
def read_dataset(filepath, columns=None, delimiter=','):
    import cudf
    columns = ['session_id', 'timestamp', 'item_id'] if columns is None else columns
    return cudf.read_csv(filepath,
                         usecols=[0, 1, 2],
                         dtype={
                             'session_id': cp.dtype('int32'),
                             'item_id': cp.dtype('int32'),
                             'timestamp': cp.dtype('O')
                         },
                         delimiter=delimiter,
                         names=columns)


In [53]:
dataset_filepath = 'archive/yoochoose-clicks.dat'

yoochoose_data = read_dataset(dataset_filepath)

In [54]:
n_rows = yoochoose_data.shape[0]
n_sessions = len(yoochoose_data['session_id'].unique())
n_items = len(yoochoose_data['item_id'].unique())
filesize = os.path.getsize(dataset_filepath)

print(f"the dataset contains {round(n_rows / 10 ** 6)}M rows, ", 
      f"with {round(n_sessions / 10 ** 6)}M ", 
      f"sessions and {round(n_items / 10 ** 3)}K items",
      f"\nOriginal file size: {round(filesize / 10 ** 6)}Mb")

the dataset contains 33M rows,  with 9M  sessions and 53K items 
Original file size: 1487Mb


### Data size:
A trained V-SKNN model needs to be able to lookup historical sessions. The data found in the dataset above (1.5GB) will have to be stored on the GPU device. 

## Data Format
Internally, the model keeps various arrays of values per `item_id` and `session_id`. It uses integer values between 1 and *N_items* to represent `item_id` (between 1 and *N_sessions* to represent`session_id`).

## Train test split
The dataset covers 183 days, we will use the first 180 days as train set, and the remaining 3 days as test set

In [55]:
yoochoose_data['day'] = yoochoose_data['timestamp'].str.slice(start=0, stop=10)

In [56]:
all_days = yoochoose_data['day'].unique()
train_days = all_days[0:180]
print(all_days)

0      2014-04-01
1      2014-04-02
2      2014-04-03
3      2014-04-04
4      2014-04-05
          ...    
178    2014-09-26
179    2014-09-27
180    2014-09-28
181    2014-09-29
182    2014-09-30
Name: day, Length: 183, dtype: object


In [57]:
train_df = yoochoose_data[yoochoose_data['day'].isin(train_days)][['session_id', 'timestamp', 'item_id']]
test_df = yoochoose_data[~yoochoose_data['day'].isin(train_days)][['session_id', 'timestamp', 'item_id']]

## Model train
training the model

In [58]:
model = CupyVsKnnModel(top_k=100, max_sessions_per_items=5000, max_item_per_session=10)

In [59]:
del yoochoose_data
gc.collect()
mempool = cp.get_default_memory_pool()
mempool.free_all_blocks()

In [60]:
start = time.time()

model.train(train_df)

end = time.time()
print(f"trained the model in {end - start} seconds")

Device memory footprint for index objects: 202.53 Mb)
trained the model in 5.978278398513794 seconds


In [61]:
del train_df
gc.collect()
mempool = cp.get_default_memory_pool()
mempool.free_all_blocks()

## Test set
To evaluate the model response time and throughput, we generate an easy to use test set. Each test session is a python list with item_id.  



In [62]:
def get_test_examples(test_set):
    test_array = test_set \
        .drop('timestamp', axis=1) \
        .groupby('session_id') \
        .agg({'item_id': 'collect'})['item_id']\
        .to_pandas()\
        .values
    return test_array

## Testing the model 


Function to test the model

In [63]:
test_sessions_array = get_test_examples(test_df)

In [64]:
def session_to_xy(items_in_session):
    return (items_in_session[-10:-1], items_in_session[-1]) if len(items_in_session) > 1 else (None, None)

In [65]:
def test_a_model(model, test_data, batch_size=1):
    
    total_hits = 0
    n_treated = 0
    hr20 = 0
    
    pbar = tqdm(test_data)
    
    predict_time = []

    for test_session in pbar:
        x, y = session_to_xy(test_session)
        if x is not None:
            x = x[-10:]
            prediction = model.predict([x] * batch_size)
            items_pred, item_scores = prediction['predicted_items'][0], prediction['scores'][0]
            predict_time.append(prediction['total_time'])
            n_treated += 1
            if len(items_pred) > 0:
                selection = cp.flip(cp.argsort(item_scores)[-20:])
                items_rec = items_pred[selection]

                if y in items_rec:
                    total_hits += 1
                    hr20 = total_hits / n_treated
                    pbar.set_postfix({'HR@20': hr20})

    time_per_iter = pbar.format_dict['elapsed'] / pbar.format_dict['n']
    avg_latency = round(sum(predict_time) / len(predict_time) * 1000, 2)
    p90_latency = round(np.quantile(np.array(predict_time), 0.9) * 1000, 2)
    throughput = round((n_treated * batch_size) / sum(predict_time))
    print(f"total inference time: {sum(predict_time)}, average latency: {avg_latency}ms average  Q90: {p90_latency}ms. throughput {throughput} items per second.")

    return time_per_iter, hr20

In [66]:
itertime_rd, hr_rd = test_a_model(model, test_sessions_array[:500], 1)

100%|██████████| 500/500 [00:02<00:00, 181.04it/s, HR@20=0.663]

total inference time: 2.2565886974334717, average latency: 6.96ms average  Q90: 10.98ms. throughput 144 items per second.


## Performance summary:

* Q90 latency: ~10ms
* Throughput: ~150 / sec
* Training time: ~6s

[smaller datasets such as this one](https://github.com/rn5l/session-rec/blob/master/data/rsc15/prepared/yoochoose-clicks-100k_train_full.txt) can obviously yield much better performances:  
With minibatches of size 20: `total inference time: 13.142018795013428, average latency: 13.16ms average  Q90: 16.88msthroughput 1520 items per second.`  
With 1 item at a time: `total inference time: 2.819638252258301, average latency: 2.82ms average  Q90: 3.17msthroughput 354 items per second.`


## Future work:
* **Optimizating further:** 
    * **Padding**: I currently use 0 padding for the value buffers, which can lead to queries taking up to 1GB of GPU memory, as well as a large number of threads. For many queries, the number of 0 values can be as high as 90%. By using a different indexing approach (more similar to the approach used for retrieving values).
    * **faster name to idx conversion**: The name to idx conversion becomes the bottleneck as the batch size increases.  Other solutions could be investigated such as [sklearn's fast dict](https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/utils/_fast_dict.pyx)
* **More paramatrization options on V-sKnn:** More weighting functions, tie-break approaches, distance functions, ... 
* **Extend algorithm:** STAN, V-STAN, .... 

